In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier


import sys
import os

sys.path.append(os.path.abspath('..'))

from src.data_processing import clean_process_data
from src.models import train_logistic_regression, train_random_forest, train_xgboost, evaluation_of_model

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
df_raw = pd.read_csv('../data/raw/heart_failure_clinical_records_dataset.csv')
df_processed = clean_process_data(df_raw)
df_processed.to_csv('../data/processed/heart_failure_scaled.csv', index=False)

In [ ]:
# I will exclude death_event to prevent data leakage

X = df_processed.drop('DEATH_EVENT', axis=1)
y = df_processed['DEATH_EVENT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 

In [ ]:
# logical regression is good choice to extract clear clinical risks.
logistic_regression_model = train_logistic_regression(X_train, y_train)

# this random forest model trains 100 decision trees, and it is good for capturing complex relationships
random_forest_model = train_random_forest(X_train, y_train)

#gradient boosting
xgboost_model = train_xgboost(X_train, y_train)

In [ ]:
print("Logistic Regression Performance:")
print(evaluation_of_model(logistic_regression_model, X_test, y_test))

In [ ]:
print("Logistic Regression Performance:")
print(evaluation_of_model(random_forest_model, X_test, y_test))

In [ ]:
print("XGBoost results: ")
print(evaluation_of_model(xgboost_model, X_test, y_test))